Решаем задачу поиска ошибки согласования рода.
Будем предсказывать род слова по его контексту.
Построен бандл.

Хотим построить батч. 

В нем должны быть признаки pymorphy. Разумеется, без рода самого слова.
Без slovnet'а потому что slovnet плохо работает на предложениях с ошибками.




Посмотрим как работает CoreExtractor - сущность, которая по слову вернет все признаки. Умеет работать с любыми индексами.

In [1]:
from sklearn import datasets
import pandas as pd
from yo_fluq_ds import *
from tg.common.ml import batched_training as bt
from tg.common import DataBundle, Loc
from tg.common.ml.batched_training import IndexedDataBundle
from tg.grammar_ru.ml.components.core_extractor.extractor import CoreExtractor
from tg.grammar_ru.ml.components.plain_context_builder import PlainContextBuilder
from tg.grammar_ru.ml.components.contextual_binding import ContextualBinding, ContextualNetworkType
from tg.common.ml.batched_training import mirrors as btm
from tg.common.ml.batched_training import Batcher
from tg.grammar_ru.common import Separator


In [2]:
db = DataBundle.load(Loc.data_cache_path/'bundles/grammatical_gender/toy')
# tiny_index = db.index[db.index.sentence_id < 10]
# tiny_db = DataBundle(**{name: df.loc[tiny_index.index]
#                      for name, df in db.data_frames.items()})
# db = tiny_db
idb = IndexedDataBundle(db.index, db)


In [3]:
Separator.Viewer().tooltip("word_id").to_html_display(
    idb.bundle.src.loc[idb.bundle.src.sentence_id == 0])


In [4]:
db.data_frames.keys()


dict_keys(['index', 'pymorphy', 'slovnet', 'src', 'syntax_closure', 'syntax_fixes', 'syntax_stats'])

In [5]:
db.index.head()


,word_id,sentence_id,label,split
sample_id,,,,
0,0,0,0,display
1,1,0,1,train
2,3,0,1,train
3,5,0,0,test
4,7,0,0,test


In [6]:
db.index.head(10).merge(
    db.data_frames['src'][['word_id', 'word']], on='word_id')


,word_id,sentence_id,label,split,word
0,0,0,0,display,Штабс-капитан
1,1,0,1,train,П
2,3,0,1,train,Н
3,5,0,0,test,Нестеров
4,7,0,0,test,днях
5,11,0,0,train,районе
6,12,0,1,test,Желтиева
7,15,0,1,train,Галиции
8,17,0,0,train,летящий
9,19,0,3,display,нашим


CoreExtractor принимает список extractor'ов и применяет их к бандлу. Потом нужно подать его ContextualBinding. TODO да?

##### Пример CoreExtractor

In [7]:
core = CoreExtractor(allow_list=['pymorphy'])  # TODO что делает allowlist
core.fit(idb)


2022-11-18 06:09:29.355894+00:00 INFO: Fitting extractor pymorphy in CoreExtractor
2022-11-18 06:09:29.633886+00:00 INFO: Success
2022-11-18 06:09:29.635888+00:00 INFO: Fitting extractor slovnet_morph in CoreExtractor
2022-11-18 06:09:29.836883+00:00 INFO: Success
2022-11-18 06:09:29.837883+00:00 INFO: Fitting extractor slovnet_syntax in CoreExtractor
2022-11-18 06:09:29.916883+00:00 INFO: Success
2022-11-18 06:09:29.917883+00:00 INFO: Fitting extractor syntax_fixes in CoreExtractor
2022-11-18 06:09:29.981900+00:00 INFO: Success
2022-11-18 06:09:29.982884+00:00 INFO: Fitting extractor syntax_stats in CoreExtractor
2022-11-18 06:09:30.071885+00:00 INFO: Success


In [8]:
core.extractors.keys()
[(e_name, e.disabled) for e_name, e in core.extractors.items()]


[('pymorphy', False),
 ('slovnet_morph', False),
 ('slovnet_syntax', False),
 ('syntax_fixes', False),
 ('syntax_stats', False)]

In [9]:
extracted = core.extract(idb)
extracted.head(3)


,pymorphy_score,pymorphy_delta_score,pymorphy_alternatives,pymorphy_POS_NOUN,pymorphy_POS_ADJF,pymorphy_POS_VERB,pymorphy_POS_PRTF,pymorphy_POS_PRTS,pymorphy_POS_ADJS,pymorphy_animacy_inan,...,syntax_fixes_root_Picked,syntax_fixes_cycle_status_No,syntax_fixes_cycle_status_Broken,syntax_fixes_cycle_status_Yes,syntax_stats_descendants_relative,syntax_stats_children,syntax_stats_descendants,syntax_stats_sentence_length,syntax_stats_up_depth,syntax_stats_down_depth
sample_id,,,,,,,,,,,,,,,,,,,,,
0,0.901142,0.949787,-0.964494,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,-0.231461,-0.065904,0.720987,1.253148,-0.880312,0.600570
1,-3.069591,-1.616324,6.928503,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,-0.314240,1.403946,0.541865,1.253148,-0.066655,-0.046574
2,-3.044473,-1.616324,6.534467,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,-0.645357,-1.177802,-1.039332,1.253148,0.510644,-1.152873


In [10]:
# list(extracted.columns)  # все признаки из pymorphy, slovnet...


##### ⚹

* Для каждого слова получили все признаки из pymorphy, slovnet...

* Нам необходим контекст слова

Будем использовать `PlainContextBuilder`.  Он построит двойной индекс.

* хотим исключить само слово из контекста, поэтому `include_zero_offset=False`

#### Пример PlainContextBuilder

In [11]:
pcb = PlainContextBuilder(include_zero_offset=False,
                          left_to_right_contexts_proportion=1)
contexts = pcb.build_context(idb, context_size=5)
contexts  # TODO у некоторых только offsset == 1


another_word_id
sample_id offset                 
0         1                     1
          2                     2
          3                     3
          4                     4
          5                     5
...                           ...
55755     2                 99930
          3                 99931
55784     1                 99974
          2                 99975
          3                 99976

[247491 rows x 1 columns]

#### ⚹

#### WordContextAssemblyPoint (~ ContextualBinding)

Задача AssemblyPoint'a (он же ContextualBinding) - порождать экстракторами батчи в том виде, в котором они будут приняты сетью. Для LSTM будет 3d-тензор. Для Plain-сети будет плоский датафрейм.

In [12]:
plain_context = ContextualBinding(
    name='plain_context',
    context_length=3,
    network_type=ContextualNetworkType.Plain,
    hidden_size=[30],
    context_builder=pcb,  # PlainContextBuilder
    # extractor=CoreExtractor(join_column='another_word_id'),
    extractor=CoreExtractor(
        allow_list=['pymorphy'], join_column='another_word_id'),
    debug=False
)


**Пояснение:**
По умолчанию CoreExtractor пытается мерджить по word_id. В нашем случае CoreExtractor отработает после создания PlainContextBuilder'ом двойного индекса. 
Для добавления признаков слов из контекста будем join'ить по столбцу another_word_id. Поэтому join_column='another_word_id'

Для создания согласованных экстракторов и сетей у AssemblyPoint есть методы create_extractor и create_network.

У такой сети на последнем слое hidden_size нейронов. Ее выход можно подать в другую сеть, которая будет выдавать вероятности классов.

In [13]:
core_extractor = plain_context.create_extractor(task=None, bundle=idb)
core_extractor.fit(idb)
# not_batch = core_extractor.extract(idb)


2022-11-18 06:09:34.036027+00:00 INFO: Fitting extractor pymorphy in CoreExtractor
2022-11-18 06:09:35.913903+00:00 INFO: Success
2022-11-18 06:09:35.915908+00:00 INFO: Fitting extractor slovnet_morph in CoreExtractor
2022-11-18 06:09:37.134524+00:00 INFO: Success
2022-11-18 06:09:37.135524+00:00 INFO: Fitting extractor slovnet_syntax in CoreExtractor
2022-11-18 06:09:38.172218+00:00 INFO: Success
2022-11-18 06:09:38.173221+00:00 INFO: Fitting extractor syntax_fixes in CoreExtractor
2022-11-18 06:09:39.087508+00:00 INFO: Success
2022-11-18 06:09:39.088501+00:00 INFO: Fitting extractor syntax_stats in CoreExtractor
2022-11-18 06:09:39.865184+00:00 INFO: Success


In [14]:
factory = plain_context.create_network_factory(
    task=None, input=None)  # None это ок. это legacy


#### Batcher

Иногда нам нужны фичи контекстов нескольких слов. Например, в задаче поиска антецедентов это местоимение и два существительных - потенциальные антецеденты. Для каждого из этих слов создадим экстрактор (для каждого экстрактора будет отдельная голова нейросети). Batcher примет список этих экстракторов и создаст батч.

`+` нам понадобится экстрагировать лейблы

In [15]:
from tg.grammar_ru.ml.components.training_task_factory import Conventions
from tg.common.ml import dft
# экстрагирует лейблы. Получается one-hot df. Лейблы пойдут в loss function.


def get_multilabel_extractor():
    label_extractor = (bt.PlainExtractor
                       .build(Conventions.LabelFrame)
                       .index()
                       .apply(take_columns=['label'], transformer=dft.DataFrameTransformerFactory.default_factory())
                       )
    return label_extractor


def test_extractor(extractor, bundle):
    extractor.fit(bundle)
    return extractor.extract(bundle)

# test_extractor(get_multilabel_extractor(), idb).head(10)


In [16]:
batcher = Batcher(batch_size=10, extractors=[
                  core_extractor, get_multilabel_extractor()])
batch = batcher.fit_extract(idb)


2022-11-18 06:10:05.228230+00:00 INFO: Fitting extractor pymorphy in CoreExtractor
2022-11-18 06:10:05.299226+00:00 INFO: Success
2022-11-18 06:10:05.300242+00:00 INFO: Fitting extractor slovnet_morph in CoreExtractor
2022-11-18 06:10:05.344226+00:00 INFO: Success
2022-11-18 06:10:05.345232+00:00 INFO: Fitting extractor slovnet_syntax in CoreExtractor
2022-11-18 06:10:05.370223+00:00 INFO: Success
2022-11-18 06:10:05.372223+00:00 INFO: Fitting extractor syntax_fixes in CoreExtractor
2022-11-18 06:10:05.397244+00:00 INFO: Success
2022-11-18 06:10:05.399240+00:00 INFO: Fitting extractor syntax_stats in CoreExtractor
2022-11-18 06:10:05.438229+00:00 INFO: Success


In [36]:
batch['index'].shape

(10, 4)

In [32]:
network = factory.create_network(task=None, input=batch)
# при создании сети необходим батч, чтобы TODO
network(batch).shape


torch.Size([10, 30])

In [18]:
# TODO tsa run_tarining. factory...


In [61]:
# batch['label']


Полные данные не помещаются в память локальной машины, но для инициализации сети нужно понять, сколько бывает классов в полных данных. Поэтому сеть инициализируется в BatchedTrainingTask, который выполняется на сервере. 

Создадим TaskFactory, отправим ее на сервер. Она подтянет данные, создаст BatchedTrainingTask и запустит его.



In [62]:
import torch


def _update_sizes_with_argument(argument_name, argument, sizes, modificator):
    if argument is None:
        return sizes
    elif isinstance(argument, torch.Tensor):
        return modificator(sizes, argument.shape[1])
    elif isinstance(argument, pd.DataFrame):
        return modificator(sizes, argument.shape[1])
    elif isinstance(argument, int):
        return modificator(sizes, argument)
    else:
        raise ValueError(
            f"Argument {argument_name} is supposed to be int, Tensor or none, but was `{argument}`")


class FullyConnectedNetwork(torch.nn.Module):
    def __init__(self,
                 sizes: List[int],
                 input: Union[None, torch.Tensor, int] = None,
                 output: Union[None, torch.Tensor, int] = None):
        super(FullyConnectedNetwork, self).__init__()
        sizes = _update_sizes_with_argument(
            'input', input, sizes, lambda s, v: [v] + s)
        sizes = _update_sizes_with_argument(
            'output', output, sizes, lambda s, v: s + [v])
        self.layers = torch.nn.ModuleList()
        for i in range(len(sizes) - 1):
            self.layers.append(torch.nn.Linear(sizes[i], sizes[i + 1]))

    def forward(self, input):
        X = input
        for layer in self.layers:
            X = layer(X)
            X = torch.sigmoid(X)
        return X


# class AssembledModel(torch.nn.Module):
#     def __init__(self, head, tail):
#         super(AssembledModel, self).__init__()
#         self.head = head
#         self.tail = tail

#     def forward(self, x):
#         x = self.head(x)
#         x = self.tail(x)
#         return x


In [75]:
from tg.common.ml.batched_training.torch.networks.extracting_network import FeedForwardNetwork

class MyNetworkFactory:
    def __init__(self, nn_head_factory):
        self.nn_head_factory = nn_head_factory

    def create_network(self, task, input):  # input is batch ~ sample
        nn_head = self.nn_head_factory.create_network(task=None, input=input)
        head_out = nn_head(input)
        hidden_size = head_out.shape[1]
        labels_count = input['label'].shape[1]
        nn_tail = FullyConnectedNetwork(
            sizes=[3], input=hidden_size, output=labels_count)
        return FeedForwardNetwork(nn_head, nn_tail, torch.nn.Softmax(dim=1))


In [76]:
assembled_network_factory = MyNetworkFactory(factory)
assembled_network = assembled_network_factory.create_network(
    task=None, input=batch)


In [77]:
mini_batch_indices = batcher.get_mini_batch_indices(mini_batch_size = 10, batch = batch)
# mini_batch = batcher.get_mini_batch(index = mini_batch_indices[0], batch = batch)
# mini_batch['index'].shape

In [78]:
# Example of target with class indices
loss = torch.nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, input)
output.backward()
# # Example of target with class probabilities
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.randn(3, 5).softmax(dim=1)
# output = loss(input, target)
# output.backward()
output

tensor(-6.4366, grad_fn=<DivBackward1>)

In [79]:
def train_loop(model, loss_fn, optimizer):
    for mini_batch_num, mini_batch_index in enumerate(mini_batch_indices):
        mini_batch = batcher.get_mini_batch(index = mini_batch_indices[mini_batch_num], batch = batch)
        # Compute prediction and loss
        pred = model(mini_batch)
        loss = loss_fn(pred, torch.tensor(mini_batch['label'].values))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # if batch % 100 == 0:
        #     loss, current = loss.item(), batch * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{num_batches:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [80]:

learning_rate = 1
optimizer = torch.optim.SGD(assembled_network.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(assembled_network, loss, optimizer)
    # test_loop(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------
Epoch 2
-------------------------------
Epoch 3
-------------------------------
Epoch 4
-------------------------------
Epoch 5
-------------------------------
Epoch 6
-------------------------------
Epoch 7
-------------------------------
Epoch 8
-------------------------------
Epoch 9
-------------------------------
Epoch 10
-------------------------------
Done!


In [60]:
mini_batch['label']

,label_0,label_1,label_3
sample_id,,,
0,1.0,0.0,0.0
1,0.0,1.0,0.0
2,0.0,1.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0
5,1.0,0.0,0.0
6,0.0,1.0,0.0


In [81]:
out = assembled_network(batch)
out.shape


torch.Size([10, 3])

In [43]:
# batch['label']

In [ ]:
import torch


class ClassificationNetwork(torch.nn.Module):
    def __init__(self, hidden_size, sample):
        super(ClassificationNetwork, self).__init__()
        self.hidden = torch.nn.Linear(sample['features'].shape[1], hidden_size)
        self.output = torch.nn.Linear(hidden_size, sample['label'].shape[1])

    def forward(self, input):
        X = input['features']
        X = torch.tensor(X.astype(float).values).float()
        X = self.hidden(X)
        X = torch.sigmoid(X)
        X = self.output(X)
        X = torch.sigmoid(X)
        return X


In [ ]:
from tg.grammar_ru.ml.components.training_task_factory import TaskFactory, Conventions
from tg.common.ml import dft


class MulticlassMetrics(bt.Metric):
    def __init__(self, add_accuracy=True, add_rating=False):
        self.add_accuracy = add_accuracy
        self.add_rating = add_rating

    def get_names(self):
        result = []
        if self.add_accuracy:
            result.append('accuracy')
        if self.add_rating:
            result.append('rating')
        return result

    def measure(self, df, _):
        prefix = 'true_label_'
        labels = []
        for c in df.columns:
            if c.startswith(prefix):
                labels.append(c.replace(prefix, ''))

        def ustack(df, prefix, cols, name):
            df = df[[prefix+c for c in cols]]
            df.columns = [c for c in cols]
            df = df.unstack().to_frame(name)
            return df

        predicted = ustack(df, 'predicted_label_', labels, 'predicted')
        true = ustack(df, 'true_label_', labels, 'true')
        df = predicted.merge(true, left_index=True,
                             right_index=True).reset_index()
        df.columns = ['label', 'sample', 'predicted', 'true']
        df = df.feed(fluq.add_ordering_column(
            'sample', ('predicted', False), 'predicted_rating'))

        match = (df.loc[df.predicted_rating ==
                 0].set_index('sample').true > 0.5)
        rating = df.loc[df.true > 0.5].set_index('sample').predicted_rating
        result = []
        if self.add_accuracy:
            result.append(match.mean())
        if self.add_rating:
            result.append(rating.mean())
        return result


In [ ]:
def get_multilabel_extractor():
    label_extractor = (bt.PlainExtractor
                       .build(Conventions.LabelFrame)
                       .index()
                       .apply(take_columns=['label'], transformer=dft.DataFrameTransformerFactory.default_factory())
                       )
    return label_extractor


In [ ]:
class ClassificationTask(TaskFactory):
    def create_task(self, data, env):
        metrics = bt.MetricPool().add(MulticlassMetrics())  # TODO epoch_count=20
        self.instantiate_default_task(
            epoch_count=1, batch_size=10000, mini_batch_size=None, metric_pool=metrics)
        self.setup_batcher(
            # data, [get_feature_extractor(), get_multilabel_extractor()])
            data, [core, get_multilabel_extractor()])
        self.setup_model(lambda _, sample: ClassificationNetwork(
            20, sample), learning_rate=1)


task = ClassificationTask()
idb = IndexedDataBundle(db.index, db)
result = task.run(idb)
# pd.DataFrame(result['output']['history']).set_index('iteration').plot()


2022-11-17 06:52:05.716220+00:00 INFO: Training starts. Info: {}
2022-11-17 06:52:05.717230+00:00 INFO: Ensuring/loading bundle. Bundle before:
2022-11-17 06:52:05.718212+00:00 INFO: Bundle loaded
{'index': {'shape': (55787, 4), 'index_name': 'sample_id', 'columns': ['word_id', 'sentence_id', 'label', 'split'], 'index': [0, 1, 2, 3, 4, '...']}, 'pymorphy': {'shape': (99977, 16), 'index_name': 'word_id', 'columns': ['normal_form', 'alternatives', 'score', 'delta_score', 'POS', '...'], 'index': [0, 1, 2, 3, 4, '...']}, 'slovnet': {'shape': (99977, 17), 'index_name': 'word_id', 'columns': ['POS', 'Animacy', 'Case', 'Gender', 'Number', '...'], 'index': [0, 1, 2, 3, 4, '...']}, 'src': {'shape': (99977, 16), 'index_name': None, 'columns': ['word_id', 'sentence_id', 'word_index', 'paragraph_id', 'word_tail', '...'], 'index': [0, 1, 2, 3, 4, '...']}, 'syntax_closure': {'shape': (260387, 4), 'index_name': 'entry_id', 'columns': ['word_id', 'syntax_parent_id', 'relation', 'distance'], 'index': [

KeyError: 'features'

setup_butcher принимает экстракторы. Передадим core? или завернем его в get_feature_extractor?